In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
# Obter e preparar o dataset

import os 
import pathlib 

_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
path_to_zip = tf.keras.utils.get_file('flower_photos', origin=_URL, untar=True, cache_dir=os.curdir)
PATH = os.path.join(os.path.dirname(path_to_zip), 'flower_photos')
data_dir = pathlib.Path(PATH)

batch_size = 32
IMG_SIZE = (180, 180)

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  shuffle=True,
  image_size=IMG_SIZE,
  batch_size=batch_size)

class_names = train_ds.class_names

In [ ]:
# https://www.tensorflow.org/tutorials/images/transfer_learning

# Transfer Learning: Opção 1
# Utilizar uma rede pré-treinada gravada em disco
# Neste caso é a Inception V1 (GoogLeNet) que ganhou o ILSVRC de 2014
# Deve existir uma pasta chamada 'inceptionv1' contendo o modelo e os pesos
# Pode obter este modelo no Moodle. Deve manter a organização das pastas e dos ficheiros

pretrained_base = tf.keras.models.load_model('inceptionv1')

# Congelar os pesos 
pretrained_base.trainable = False


In [ ]:
pretrained_base.summary()


In [ ]:
# Criar o modelo completo
# Poderia ser adicionada u ma camada de data augmentation ao início

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = keras.Sequential([
    keras.Input(shape=(128, 128, 3)),
    pretrained_base,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(5, activation='softmax')
])

In [ ]:
# Compilar o modelo

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
model.summary()

In [ ]:
# Preparar o Dataset: ajustar o tamanho das imagens e aplicar operações de pré-processamento
size = (128, 128)

train_ds_IncV1 = train_ds.map(lambda x, y: (tf.image.resize(x, size), y))
val_ds_IncV1 = val_ds.map(lambda x, y: (tf.image.resize(x, size), y))

train_ds_IncV1 = train_ds_IncV1.cache().prefetch(1)
val_ds_IncV1 = val_ds_IncV1.cache().prefetch(1)

normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

train_ds_IncV1 = train_ds_IncV1.map(lambda x, y: (normalization_layer(x), y))
val_ds_IncV1 = val_ds_IncV1.map(lambda x, y: (normalization_layer(x), y))



In [ ]:
# Treinar

history = model.fit(
    train_ds_IncV1,
    validation_data=val_ds_IncV1,
    epochs=15,
)

In [ ]:
# Visualizar os resultados
# Visualização da evolução da accurary e da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# Descongelar os pesos da componente pré-treinada

pretrained_base.trainable = True

In [ ]:
# Voltar a compilar

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

model.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Treinar mais um pouco 
# Agora vai demorar mais a completar cada época de treino, uma vez que todos os pesos estão a ser ajustados
# O número de épocas é baixo e a CNN ainda não deverá ter estabilizado no final do treino (confirmar no gráfico)

history = model.fit(
    train_ds_IncV1,
    validation_data=val_ds_IncV1,
    epochs=10,
)

In [ ]:
# Visualizar os resultados
# Visualização da evolução da accurary e da loss

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
# https://keras.io/api/applications/

# Transfer Learning: Opção 2
# Usar um dos modelos pré-treinados disponíveis no Keras
# Neste caso é a ResNet50, uma variante simplificada da CNN que ganhou o ILSVRC de 2015

myResNet = keras.applications.resnet50.ResNet50(weights="imagenet", include_top=False)

myResNet.trainable = False

In [ ]:
# Criar o modelo completo
# A camada de global average pooling transforma a informação para uma estrutura linear 
modelR = keras.Sequential([
    keras.Input(shape=(224, 224, 3)),
    myResNet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(5, activation='softmax')
])

In [ ]:
myResNet.summary()

In [ ]:
modelR.summary()

In [ ]:
# Pré-processammento
# Necessário chamar método tf.keras.applications.resnet.preprocess_input para preparar o input

def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = keras.applications.resnet.preprocess_input(resized_image)
    return final_image, label

#normalization_layer = keras.layers.experimental.preprocessing.Rescaling(1./255)

#train_ds_RN = train_ds.map(lambda x, y: (normalization_layer(x), y))
#val_ds_RN = val_ds.map(lambda x, y: (normalization_layer(x), y))

train_ds_RN = train_ds.map(preprocess).cache().prefetch(1)
val_ds_RN = val_ds.map(preprocess).cache().prefetch(1)

In [ ]:
# Compilar Modelo R

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

modelR.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:
# Treinar (vai demorar se não estiver a ser usada uma GPU ou TPU)


history = modelR.fit(
    train_ds_RN,
    validation_data=val_ds_RN,
    epochs=15,
)

In [ ]:
# Visualizar os resultados
# Visualização da evolução da accurary e da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()

In [ ]:
# Descongelar os pesos da componente pré-treinada

myResNet.trainable = True

In [ ]:
# Voltar a compilar

optimizer = tf.keras.optimizers.Adam(epsilon=0.01)

modelR.compile(
    optimizer=optimizer,
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [ ]:

# Treinar mais um pouco 
# Agora vai demorar muito mais tempo a completar cada época de treino, uma vez que todos os pesos estão a ser ajustados
# O número de épocas é baixo e a CNN ainda não deverá ter estabilizado no final do treino (confirmar no gráfico)

history = modelR.fit(
    train_ds_RN,
    validation_data=val_ds_RN,
    epochs=15,
)

In [ ]:
# Visualizar os resultados
# Visualização da evolução da accurary e da loss

import pandas as pd 
history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['accuracy', 'val_accuracy']].plot()